<a href="https://colab.research.google.com/github/raksnotes/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [73.4 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,544 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 http://archiv

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-03-20 20:34:20--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.18MB/s    in 0.8s    

2023-03-20 20:34:22 (1.18 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_FR_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         FR|      14952|R32VYUWDIB5LKE|0552774294|     362925721|    The God Delusion|               Books|          5|            0|          0|   N|                Y|a propos de ce livre|je conseille fort...| 2013-02-13|
|         FR|      14952|R3CCMP4EV6HAVL|B004GJXQ20|     268067011|A Game of Thrones...|Digital_Ebook_Pur

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [43]:
# Create the customers_table DataFrame
# Use the groupby() function on the customer_id column of the DataFrame you created in Step 7
# Count all the customer ids using the agg() function by chaining it to the groupby() function. After you use this function, a new column will be created, count(customer_id).
# Rename the count(customer_id) column using the withColumnRenamed() function so it matches the schema for the customers_table in pgAdmin.
customers_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      26425|             1|
|      29814|             1|
|      78440|             2|
|      90550|             1|
|      99817|             1|
|    2846240|             1|
|    2966808|             1|
|    3323007|             1|
|    3458442|             1|
|    3497479|             1|
|    3498767|             1|
|    3511947|             1|
|    3945593|             1|
|    4049807|             1|
|    4214333|             1|
|    4262489|             2|
|    4423902|             1|
|    4660180|             2|
|    4667529|             3|
|    4838481|             2|
+-----------+--------------+
only showing top 20 rows



In [11]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00GIGGS6A|      Huion H610 PRO|
|B00XO12REY|  The Monsanto Years|
|0141976144|Wreck This Journa...|
|B00KT2Z9WS|Rival Knights (Ki...|
|B000A3X4KU|Closer : Entre ad...|
|B00R5DXFS4|50 nuances de Gre...|
|B00HLBDYZO|Figurine - Sailor...|
|B00B1D6ZWC|Schleich - 14699 ...|
|B00C59AVCG|SHARK Montre de s...|
|B00005LL1B|           Rio Bravo|
|B00FA4ZE66|Godzilla [Blu-ray...|
|B00063EMJ6|Neil Young. Great...|
|B005HH4YGW|Ugly Betty, saiso...|
|B00HQF9WEC|Vol.2-Formula [Im...|
|1408850753|Harry Potter Boxe...|
|B001AW156U|Gâchettes pour Ma...|
|B005HUBCOQ|Cowboys & Envahis...|
|B00HZ63GH6|Man of Steel - Bl...|
|B002CXG7IW|       Ali [Blu-ray]|
|B00BSVSSLI|Batman, le défi -...|
+----------+--------------------+
only showing top 20 rows



In [14]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# To create the review_id_table, use the select() function to select the columns that are in the review_id_table in pgAdmin
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R32VYUWDIB5LKE|      14952|0552774294|     362925721| 2013-02-13|
|R3CCMP4EV6HAVL|      14952|B004GJXQ20|     268067011| 2014-08-03|
|R14NAE6UGTVTA2|      17564|B00GIGGS6A|     256731097| 2015-07-07|
|R2E7QEWSC6EWFA|      18940|B00CW7KK9K|     977480037| 2014-06-16|
|R26E6I47GQRYKR|      20315|B002L6SKIK|     827187473| 2013-06-10|
|R1RJMTSNCKB9LP|      20842|B00004YLIU|     678427290| 2013-04-15|
|R2P2XF84YELQBZ|      20913|B00AYHK7RU|     108403123| 2013-06-07|
| RJKULSX2Y5R07|      21490|B00CJ3V5UK|     252503117| 2014-11-07|
|R3UYE0U7SQCI8Q|      24196|B000FUM0TE|      48021829| 2012-11-16|
|R1TKJ7XFS3RDEB|      24196|B000UMXCTY|     181554537| 2013-04-23|
|R3S9JNS21QDBXP|      24196|B0044JV1K6|     386772628| 2013-08-02|
|R2Y4O06QMOGHD0|      24196|2226249672|     753674225| 2013-10

In [16]:
# Create the vine_table. DataFrame
# To create the vine_table, use the select() function to select only the columns that are in the vine_table in pgAdmin 
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R32VYUWDIB5LKE|          5|            0|          0|   N|                Y|
|R3CCMP4EV6HAVL|          5|            0|          0|   N|                Y|
|R14NAE6UGTVTA2|          3|            1|          3|   N|                Y|
|R2E7QEWSC6EWFA|          4|            0|          1|   N|                Y|
|R26E6I47GQRYKR|          2|            3|          5|   N|                N|
|R1RJMTSNCKB9LP|          2|            0|          0|   N|                Y|
|R2P2XF84YELQBZ|          5|            0|          3|   N|                Y|
| RJKULSX2Y5R07|          5|            0|          0|   N|                Y|
|R3UYE0U7SQCI8Q|          5|            2|          3|   N|                Y|
|R1TKJ7XFS3RDEB|          5|            0|          1|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [36]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.crhidxlus5cr.us-east-2.rds.amazonaws.com:5432/reviews"
config = {"user":"postgresRK", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [37]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [38]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [39]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [40]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)